In [ ]:
import sys, os
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolAgentAction, ToolsAgentOutputParser
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.messages.ai import add_ai_message_chunks
from langchain_core.tools import tool
from langchain_core.tools.structured import StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

In [3]:
# Автоперезагрузка файлов
%load_ext autoreload
%autoreload 2

# Set env
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"

In [6]:
# Определим функции
def add_tool(a: int, b: int) -> int:
    """Складывает два числа 'a' и 'b'"""
    return a + b

def multiply_tool(a: int, b: int) -> int:
    """Перемножает два числа 'a' и 'b'"""
    return a * b

def magic_function(a: int) -> int:
    """Applies a magic function to an input."""
    return a + 5

class AI:
    
    def __init__(self):
        self.tools = [
            tool(add_tool),
            tool(multiply_tool),
            tool(magic_function),
        ]
    
    def get_llm(self, name):
        llm_answer = ChatOllama(
            base_url="http://database_ollama:11434",
            model=name,
            temperature=0.5,
            streaming=True,
        )
        llm_with_tools = llm_answer.bind_tools(self.tools)
        return llm_with_tools

    def find_tool(self, action):
        for tool in self.tools:
            if tool.name == action.tool:
                return tool
    
    def run_tools(self, items):
        
        response = []
        for action in items:
            tool = self.find_tool(action)
            observation = None
            
            if tool:
                print("Run tool " + action.log)
                observation = tool.run(action.tool_input)
                print("Answer " + str(observation))
            
            response.append((action, observation))
        
        return response

    def send_question(self, llm, prompt_value):
        chunks = []
        response = llm.stream(prompt_value)
        for chunk in response:
            print(chunk.content, end="", flush=True)
            chunks.append(chunk)
        
        # Get tools
        tool_calls = []
        for chunk in chunks:
            if hasattr(chunk, "tool_calls") and chunk.tool_calls:
                tool_calls.extend(chunk.tool_calls)
        
        # Merge chunks
        chunk = None
        if len(chunks) > 0:
            chunk = chunks[0]
            if len(chunks) > 1:
                chunk = add_ai_message_chunks(chunk, *chunks[1:])
        
        # Build mesage
        message = AIMessage(
            content=chunk.content,
            tool_calls=tool_calls,
            additional_kwargs=chunk.additional_kwargs if chunk is not None else {},
            response_metadata=chunk.response_metadata if chunk is not None else {},
            usage_metadata=chunk.usage_metadata if chunk is not None else None
        )
        return message
    
    def get_tools_answer(self, answer):
        items = []
        for item in answer.tool_calls:
            log = item["name"] + " with args " + str(item["args"])
            action = ToolAgentAction(
                tool=item["name"],
                tool_input=item["args"],
                log=log,
                message_log=[answer],
                tool_call_id=item["id"],
            )
            items.append(action)
        return items
    
    def send_message(self, llm, query):
        
        step = 1
        response_step = []
        tool_calling_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "Ты консультант для программистов. Отвечай на русском языке в деловом стиле. Если задача требует вычислений или инструментов, используй доступные функции."),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]
        )
        while True:
            prompt = {
                "input": query,
                "intermediate_steps": response_step,
                "agent_scratchpad": format_to_tool_messages(response_step)
            }
            prompt_value = tool_calling_prompt.invoke(prompt)
            
            print ("Step " + str(step))
            answer = self.send_question(llm, prompt_value)
            display(answer)
            if len(answer.tool_calls) == 0:
                break
            
            tools = self.get_tools_answer(answer)
            response_step.extend(self.run_tools(tools))
            step = step + 1
        
        return answer

# Query
#query = "Выполни волшебную функцию над числом 5. Ответ:"
#query = "Сначала вычисли 5 + 7, а затем выполни волшебную функцию над результатом сложения. Ответ:"
#query = "Выполни волшебную функцию над числом 5, а затем результат передай еще раз в волшебную функцию. Ответ:"
#query = "Сколько будет 2 + 3?"
query = "Напиши простую функцию сортировки массива на javascript"

# Get answer
ai = AI()
model = ai.get_llm("qwen2.5:3b")
answer = ai.send_message(model, query)
print("\nOk")

Step 1
Для создания простой функции сортировки массива на JavaScript, можно использовать метод `sort()` объекта массива. Однако, для сортировки по возрастанию или убыванию и обеспечения правильной работы с датами, лучше использовать сравнительную функцию.

Вот пример простого метода сортировки массива по возрастанию:

```javascript
function simpleSort(arr) {
    return arr.sort((a, b) => a - b);
}
```

Если вам нужно выполнить более сложные операции при сортировке (например, сортировка не только числовых значений), можно использовать следующий пример:

```javascript
function customSort(arr) {
    return arr.sort((a, b) => a.localeCompare(b));
}
```

Этот метод будет работать для строковых значений, отсортируя их в алфавитном порядке.

Выберите подходящий способ сортировки в зависимости от ваших требований.

AIMessage(content='Для создания простой функции сортировки массива на JavaScript, можно использовать метод `sort()` объекта массива. Однако, для сортировки по возрастанию или убыванию и обеспечения правильной работы с датами, лучше использовать сравнительную функцию.\n\nВот пример простого метода сортировки массива по возрастанию:\n\n```javascript\nfunction simpleSort(arr) {\n    return arr.sort((a, b) => a - b);\n}\n```\n\nЕсли вам нужно выполнить более сложные операции при сортировке (например, сортировка не только числовых значений), можно использовать следующий пример:\n\n```javascript\nfunction customSort(arr) {\n    return arr.sort((a, b) => a.localeCompare(b));\n}\n```\n\nЭтот метод будет работать для строковых значений, отсортируя их в алфавитном порядке.\n\nВыберите подходящий способ сортировки в зависимости от ваших требований.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-07-26T13:27:01.008871344Z', 'done': True, 'done_reason': 'stop',


Ok


In [ ]:
ai = AI()
model = ai.get_llm("qwen2.5:3b")

# Query
query = "Сколько будет 5 + 7. Ответ:"

# Get tool
tool_calling_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты консультант для программистов. Отвечай на русском языке в деловом стиле. Если задача требует вычислений или инструментов, используй доступные функции."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
prompt = tool_calling_prompt.invoke({"input": query, "intermediate_steps": [], "agent_scratchpad": []})
response = model.invoke(prompt)
display(response)